----
#Pandas DataFrame Data Structure Continued
----

##Indexing DataFrames

As we've seen, both Series and DataFrames can have indices applied to them. The index is essentially a row level label, and in pandas the rows correspond to axis zero. 

Indices can be either autogenerated, such as when we create a new Series without an index, in which case we get numeric values, or they can be set explicitly, like when we use the dictionary object to create the series, or when we loaded data from the CSV file and set appropriate parameters. Another option for setting an index is to use the `set_index()` function. This function takes a list of columns and promotes those columns to an index. In this lecture we'll explore more about how indexes work in pandas.

**Important note: the `set_index()` function is a destructive process that doesn't keep the current index. If you want to keep the current index, you need to manually create a new column and copy into it values from the index attribute.**

Let's import pandas and our admissions dataset.


In [115]:
# Again, to access the file, you need to mount the drive. 
from google.colab import drive
drive.mount('/content/drive')
#!ls /content/drive/My\ Drive/Applied\ Data\ Science\ in\ Python/datasets/  # Running a line with a "!" in the start is identical to running a bash script

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [116]:
import pandas as pd

df = pd.read_csv("/content/drive/My Drive/Applied Data Science in Python/datasets/Admission_Predict.csv", index_col="Serial No.")

df.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,324,107,4,4.0,4.5,8.87,1,0.76
3,316,104,3,3.0,3.5,8.00,1,0.72
4,322,110,3,3.5,2.5,8.67,1,0.80
5,314,103,2,2.0,3.0,8.21,0,0.65


Let's say that we don't want to index the DataFrame by "Serial Number", but instead by "Chance of Admit". 

First, we need to preserve the serial number into a new column. We can do this using the `.index` attribute to save the index on a new column labeled "Serial Number". Then we can use `set_index` to set the index of the DataFrame to "Chance of Admit".


In [ ]:
# We copy the indexed data into its own column
df['Serial Number'] = df.index
# Then we set the index to another column
df = df.set_index('Chance of Admit ')
df.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Serial Number
Chance of Admit,,,,,,,,
0.92,337,118,4,4.5,4.5,9.65,1,1
0.76,324,107,4,4.0,4.5,8.87,1,2
0.72,316,104,3,3.0,3.5,8.00,1,3
0.80,322,110,3,3.5,2.5,8.67,1,4
0.65,314,103,2,2.0,3.0,8.21,0,5


You'll see that when we create a new index from an existing column the index gets a name, which is the original name of the column.

We can also "reset" the index and get rid of the index completely by calling the function `reset_index()`. This promotes the old index into a new column and creates the default numbered index.

In [ ]:
df = df.reset_index()
df.head()

,Chance of Admit,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Serial Number
0,0.92,337,118,4,4.5,4.5,9.65,1,1
1,0.76,324,107,4,4.0,4.5,8.87,1,2
2,0.72,316,104,3,3.0,3.5,8.00,1,3
3,0.80,322,110,3,3.5,2.5,8.67,1,4
4,0.65,314,103,2,2.0,3.0,8.21,0,5


###Multi-level Indexing

One nice feature of Pandas is **multi-level indexing**. This is similar to composite keys in relational database systems. To create a multi-level index, we simply call `set_index()` and give it a list of columns that we're interested in promoting to an index.

Pandas will search through these in order, finding the distinct data and form composite indices. A good example of this is often found when dealing with geographical data which is sorted by regions or demographics.

Let's change data sets and look at some census data for a better example. Let's upload the file "census.csv" containing data from the United States Census Bureau. In particular, it contains a breakdown of the population at the US county level. It's a great example of how different kinds of data sets might be formatted when you're trying to clean them.


In [ ]:
# Let's import and see what the data looks like
df = pd.read_csv('/content/drive/My Drive/Applied Data Science in Python/datasets/census.csv')
df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,NPOPCHG_2010,NPOPCHG_2011,NPOPCHG_2012,NPOPCHG_2013,NPOPCHG_2014,NPOPCHG_2015,BIRTHS2010,BIRTHS2011,BIRTHS2012,BIRTHS2013,BIRTHS2014,BIRTHS2015,DEATHS2010,DEATHS2011,DEATHS2012,DEATHS2013,DEATHS2014,DEATHS2015,NATURALINC2010,NATURALINC2011,NATURALINC2012,NATURALINC2013,NATURALINC2014,NATURALINC2015,INTERNATIONALMIG2010,...,RESIDUAL2013,RESIDUAL2014,RESIDUAL2015,GQESTIMATESBASE2010,GQESTIMATES2010,GQESTIMATES2011,GQESTIMATES2012,GQESTIMATES2013,GQESTIMATES2014,GQESTIMATES2015,RBIRTH2011,RBIRTH2012,RBIRTH2013,RBIRTH2014,RBIRTH2015,RDEATH2011,RDEATH2012,RDEATH2013,RDEATH2014,RDEATH2015,RNATURALINC2011,RNATURALINC2012,RNATURALINC2013,RNATURALINC2014,RNATURALINC2015,RINTERNATIONALMIG2011,RINTERNATIONALMIG2012,RINTERNATIONALMIG2013,RINTERNATIONALMIG2014,RINTERNATIONALMIG2015,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,4801108,4816089,4830533,4846411,4858979,5034,15947,14981,14444,15878,12568,14226,59689,59062,57938,58334,58305,11089,48811,48357,50843,50228,50330,3137,10878,10705,7095,8106,7975,1357,...,677,-573,1135,116185,116212,115560,115666,116963,119088,119599,12.453020,12.282581,12.012080,12.056286,12.014973,10.183524,10.056360,10.541099,10.380963,10.371556,2.269496,2.226220,1.470981,1.675322,1.643417,1.027720,1.019840,1.002216,1.142716,1.179963,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,55253,55175,55038,55290,55347,89,593,-78,-137,252,57,151,636,615,574,623,600,152,507,558,583,504,467,-1,129,57,-9,119,133,33,...,22,-10,45,455,455,455,455,455,455,455,11.572789,11.138479,10.416194,11.293597,10.846281,9.225478,10.106133,10.579514,9.136393,8.442022,2.347311,1.032347,-0.163320,2.157204,2.404259,0.363924,0.289782,0.290347,0.326300,0.343466,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,186659,190396,195126,199713,203709,928,3466,3737,4730,4587,3996,517,2187,2092,2160,2186,2240,532,1825,1879,1902,2044,1992,-15,362,213,258,142,248,69,...,91,434,58,2307,2307,2307,2249,2304,2308,2309,11.826352,11.096524,11.205586,11.072868,11.104997,9.868812,9.966716,9.867141,10.353587,9.875515,1.957540,1.129809,1.338445,0.719281,1.229482,1.011215,0.912334,0.881921,1.073855,1.095627,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,27226,27159,26973,26815,26489,-116,-115,-67,-186,-158,-326,70,335,300,283,260,269,128,319,291,294,310,309,-58,16,9,-11,-50,-40,2,...,19,-1,-5,3193,3193,3382,3388,3389,3353,3352,12.278483,11.032454,10.455923,9.667584,10.093051,11.692048,10.701480,10.862337,11.526735,11.593877,0.586435,0.330974,-0.406414,-1.859151,-1.500825,-0.146609,-0.257424,-0.110840,-0.074366,0.000000,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,22733,22642,22512,22549,22583,-58,-128,-91,-130,37,34,44,266,245,259,247,253,34,278,237,281,211,223,10,-12,8,-22,36,30,2,...,14,-16,-21,2224,2224,2224,2224,2224,2233,2236,11.668202,10.798898,11.471852,10.962917,11.211557,12.194587,10.446281,12.446295,9.365083,9.882124,-0.526385,0.352617,-0.974443,1.597834,1.329434,0.438654,0.705234,0.797272,0.932070,0.930604,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In this data set there are two summarized levels ("SUMLEV" column): one that contains summary data for the whole state (SUMLEV==40), and one that contains summary data for each county in each state (SUMLEV==50). 

Let's say I want to see a list of all the unique values in a given column. For example, in this DataFrame, we can see all possible unique values for the summarized level are using the `.unique()` function on the DataFrame. This is similar to the SQL distinct operator.


In [ ]:
# Here we can run .unique() on the sum level of our current DataFrame 
df['SUMLEV'].unique()

array([40, 50])

As you can see, there are only two different values: 40 and 50.

Let's exclude all of the rows that are summaries at the state level and just keep the county data. 

In [ ]:
df = df[df['SUMLEV'] == 50]
df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,NPOPCHG_2010,NPOPCHG_2011,NPOPCHG_2012,NPOPCHG_2013,NPOPCHG_2014,NPOPCHG_2015,BIRTHS2010,BIRTHS2011,BIRTHS2012,BIRTHS2013,BIRTHS2014,BIRTHS2015,DEATHS2010,DEATHS2011,DEATHS2012,DEATHS2013,DEATHS2014,DEATHS2015,NATURALINC2010,NATURALINC2011,NATURALINC2012,NATURALINC2013,NATURALINC2014,NATURALINC2015,INTERNATIONALMIG2010,...,RESIDUAL2013,RESIDUAL2014,RESIDUAL2015,GQESTIMATESBASE2010,GQESTIMATES2010,GQESTIMATES2011,GQESTIMATES2012,GQESTIMATES2013,GQESTIMATES2014,GQESTIMATES2015,RBIRTH2011,RBIRTH2012,RBIRTH2013,RBIRTH2014,RBIRTH2015,RDEATH2011,RDEATH2012,RDEATH2013,RDEATH2014,RDEATH2015,RNATURALINC2011,RNATURALINC2012,RNATURALINC2013,RNATURALINC2014,RNATURALINC2015,RINTERNATIONALMIG2011,RINTERNATIONALMIG2012,RINTERNATIONALMIG2013,RINTERNATIONALMIG2014,RINTERNATIONALMIG2015,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,55253,55175,55038,55290,55347,89,593,-78,-137,252,57,151,636,615,574,623,600,152,507,558,583,504,467,-1,129,57,-9,119,133,33,...,22,-10,45,455,455,455,455,455,455,455,11.572789,11.138479,10.416194,11.293597,10.846281,9.225478,10.106133,10.579514,9.136393,8.442022,2.347311,1.032347,-0.163320,2.157204,2.404259,0.363924,0.289782,0.290347,0.326300,0.343466,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,186659,190396,195126,199713,203709,928,3466,3737,4730,4587,3996,517,2187,2092,2160,2186,2240,532,1825,1879,1902,2044,1992,-15,362,213,258,142,248,69,...,91,434,58,2307,2307,2307,2249,2304,2308,2309,11.826352,11.096524,11.205586,11.072868,11.104997,9.868812,9.966716,9.867141,10.353587,9.875515,1.957540,1.129809,1.338445,0.719281,1.229482,1.011215,0.912334,0.881921,1.073855,1.095627,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,27226,27159,26973,26815,26489,-116,-115,-67,-186,-158,-326,70,335,300,283,260,269,128,319,291,294,310,309,-58,16,9,-11,-50,-40,2,...,19,-1,-5,3193,3193,3382,3388,3389,3353,3352,12.278483,11.032454,10.455923,9.667584,10.093051,11.692048,10.701480,10.862337,11.526735,11.593877,0.586435,0.330974,-0.406414,-1.859151,-1.500825,-0.146609,-0.257424,-0.110840,-0.074366,0.000000,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,22733,22642,22512,22549,22583,-58,-128,-91,-130,37,34,44,266,245,259,247,253,34,278,237,281,211,223,10,-12,8,-22,36,30,2,...,14,-16,-21,2224,2224,2224,2224,2224,2233,2236,11.668202,10.798898,11.471852,10.962917,11.211557,12.194587,10.446281,12.446295,9.365083,9.882124,-0.526385,0.352617,-0.974443,1.597834,1.329434,0.438654,0.705234,0.797272,0.932070,0.930604,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57373,57711,57776,57734,57658,57673,51,338,65,-42,-76,15,183,744,710,646,618,603,133,570,592,585,589,590,50,174,118,61,29,13,5,...,-22,-14,53,489,489,489,489,489,489,489,12.929686,12.295756,11.185179,10.711314,10.456859,9.905808,10.252236,10.128993,10.208680,10.231421,3.023878,2.043520,1.056186,0.502634,0.225438,0.052136,0.329041,0.346290,0.485302,0.485559,1.807375,-1.177622,-1.748766,-2.062535,-1.369970,1.859511,-0.848580,-1.402476,-1.577232,-0.884411


While this data set is interesting for a number of different reasons, let's reduce the data that we're going to look at to just the total population estimates and the total number of births. We can do this by creating a list of column names that we want to keep then project those and assign the resulting DataFrame to our `df` variable.

In [ ]:
columns_to_keep = ['STNAME','CTYNAME','BIRTHS2010','BIRTHS2011','BIRTHS2012','BIRTHS2013',
                   'BIRTHS2014','BIRTHS2015','POPESTIMATE2010','POPESTIMATE2011',
                   'POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']
df = df[columns_to_keep]
df.head()

,STNAME,CTYNAME,BIRTHS2010,BIRTHS2011,BIRTHS2012,BIRTHS2013,BIRTHS2014,BIRTHS2015,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015
1,Alabama,Autauga County,151,636,615,574,623,600,54660,55253,55175,55038,55290,55347
2,Alabama,Baldwin County,517,2187,2092,2160,2186,2240,183193,186659,190396,195126,199713,203709
3,Alabama,Barbour County,70,335,300,283,260,269,27341,27226,27159,26973,26815,26489
4,Alabama,Bibb County,44,266,245,259,247,253,22861,22733,22642,22512,22549,22583
5,Alabama,Blount County,183,744,710,646,618,603,57373,57711,57776,57734,57658,57673


The US Census data breaks down population estimates by state and county. We can load the data and set the index to be a combination of the state and county values and see how pandas handles it in a DataFrame. We do this by creating a list of the column identifiers/labels we want to have indexed. We then pass this list as a parameter to `set_index()`. The result will be a dual index, first the state name and second the county name.

In [ ]:
df = df.set_index(['STNAME','CTYNAME'])
df.head()

                        BIRTHS2010  ...  POPESTIMATE2015
STNAME  CTYNAME                     ...                 
Alabama Autauga County         151  ...            55347
        Baldwin County         517  ...           203709
        Barbour County          70  ...            26489
        Bibb County             44  ...            22583
        Blount County          183  ...            57673

[5 rows x 12 columns]


So how do we query this DataFrame?

As we saw previously, the `.loc` attribute of the DataFrame can take multiple arguments. It could query both the row and the columns. When you use a MultiIndex, you must provide the arguments in order by the level you wish to query. Inside of the index, each column is called a level and the outermost column is level zero. 

If we want to see the population results from Champaign County in the state of Illinois, the first argument would be Illinois and the second would be Champaign County

In [ ]:
df.loc['Illinois', 'Champaign County']

BIRTHS2010            602
BIRTHS2011           2333
BIRTHS2012           2423
BIRTHS2013           2352
BIRTHS2014           2414
BIRTHS2015           2410
POPESTIMATE2010    201489
POPESTIMATE2011    202681
POPESTIMATE2012    204080
POPESTIMATE2013    205785
POPESTIMATE2014    207424
POPESTIMATE2015    208861
Name: (Illinois, Champaign County), dtype: int64

Let's say we are interested in comparing two counties: Champaign and Douglas County. We can pass a list of tuples describing the indices we wish to query into `loc`. Since we have a MultiIndex of two values, the state and the county, we need to provide two values as each element of our filtering list. Each tuple should have two elements, the first element being the first index and the second element being the second index.

Therefore, in this case, we will have a list of two tuples, in each tuple, the first element is Illinois, and the second element is either Champaign County or Douglas County.

In [ ]:
df.loc[ [('Illinois', 'Champaign County'),
         ('Illinois', 'Douglas County')] ]

BIRTHS2010  ...  POPESTIMATE2015
STNAME   CTYNAME                       ...                 
Illinois Champaign County         602  ...           208861
         Douglas County            64  ...            19823

[2 rows x 12 columns]

Okay so that's how hierarchical indices work in a nutshell. They're a special part of the pandas library which I think can make management and reasoning about data easier. Of course hierarchical labeling isn't just for rows. 
For example, you can transpose this matrix and now have hierarchical column labels, and projecting a single column which has these labels works exactly the way you would expect it to. 

Now, in reality, I don't tend to use hierarchical indicies very much, and instead just keep everything as columns and manipulate those. But, it's a 
unique and sophisticated aspect of pandas that is useful to know, especially if viewing your data in a tabular form.

##Missing Values

We've seen a preview of how Pandas handles missing values using the `None` type and NumPy `NaN` values. Missing values are pretty common in data cleaning activities, and can be there for any number of
reasons. Here are a few of those reasons:

1. For instance, if you are running a survey and a respondant didn't answer a question, the missing value is actually an omission. This kind of missing data is called **Missing at Random** if there are other variables that might be used to predict the variable which is missing. For example, when working on surveys I often find that missing data, say the interest in being involved in a follow up study, often has some correlation with another data field, like gender or ethnicity. 

2. If there is no relationship to other variables, then we call this data **Missing Completely at Random (MCAR)**.

3. Finally, data might be missing because it wasn't collected, either by the process responsible for collecting that data, such as a researcher, or because it wouldn't make sense if it were collected. 

This last example is extremely common when you start joining DataFrames together from multiple sources, such as joining a list of people at a university with a list of offices in the university (students generally don't have offices).

Let's look at some ways of handling missing data in pandas.

In [ ]:
# Let's import pandas
import pandas as pd

Pandas is pretty good at detecting missing values directly from underlying data formats, like CSV files. Although most missing values are often formatted as `NaN`, `NULL`, `None`, or `N/A`, sometimes missing values are not labeled so clearly. For example, I've worked with social scientists who regularly used the value of `99` in binary categories to indicate a missing value. The pandas `read_csv()` function has a parameter called `na_values` to let us specify the form of missing values. It allows scalar, string, list, or dictionaries to be used.

In [ ]:
# Let's load data from a file called class_grades.csv
df = pd.read_csv('/content/drive/My Drive/Applied Data Science in Python/datasets/class_grades.csv')
df

,Prefix,Assignment,Tutorial,Midterm,TakeHome,Final
0,5,57.14,34.09,64.38,51.48,52.50
1,8,95.05,105.49,67.50,99.07,68.33
2,8,83.70,83.17,NaN,63.15,48.89
3,7,NaN,NaN,49.38,105.93,80.56
4,8,91.32,93.64,95.00,107.41,73.89
...,...,...,...,...,...,...
94,8,NaN,103.71,45.00,93.52,61.94
95,7,NaN,80.54,41.25,93.70,39.72
96,8,89.94,102.77,87.50,90.74,87.78
97,7,95.60,76.13,66.25,99.81,85.56


We can actually use the function `.isnull()` to create a boolean mask of the whole dataframe. This effectively broadcasts the `.isnull()` function to every cell of data.

In [ ]:
mask=df.isnull()
mask.head(10)

,Prefix,Assignment,Tutorial,Midterm,TakeHome,Final
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,True,False,False
3,False,True,True,False,False,False
4,False,False,False,False,False,False
5,False,False,False,False,False,False
6,False,False,False,False,False,False
7,False,False,False,False,True,False
8,False,False,False,False,False,False
9,False,False,False,False,False,False


This can be useful for processing rows based on certain columns of data. 

Another useful operation is to be able to drop all of those rows which have *any* missing data, which can be done with the `.dropna()` function.


In [ ]:
df.dropna().head(10)

,Prefix,Assignment,Tutorial,Midterm,TakeHome,Final
0,5,57.14,34.09,64.38,51.48,52.50
1,8,95.05,105.49,67.50,99.07,68.33
4,8,91.32,93.64,95.00,107.41,73.89
5,7,95.00,92.58,93.12,97.78,68.06
6,8,95.05,102.99,56.25,99.07,50.00
8,8,84.26,93.10,47.50,18.52,50.83
9,7,90.10,97.55,51.25,88.89,63.61
10,7,80.44,90.20,75.00,91.48,39.72
12,8,97.16,103.71,72.50,93.52,63.33
13,7,91.28,83.53,81.25,99.81,92.22


Note how the rows indexed with 2, 3, 7, and 11 are now gone. Another handy function that Pandas has for working with missing values is the filling function, `fillna()`. This function takes a number of parameters.

You could pass in a single value (a.k.a. a scalar value) to change all of the missing data to one value. This isn't really applicable in this case, but it's a pretty common use case.

Let's say we want to fill all missing values with 0, we would use `fillna`.


In [ ]:
df.fillna(0, inplace=True)
df.head(10)

,Prefix,Assignment,Tutorial,Midterm,TakeHome,Final
0,5,57.14,34.09,64.38,51.48,52.50
1,8,95.05,105.49,67.50,99.07,68.33
2,8,83.70,83.17,0.00,63.15,48.89
3,7,0.00,0.00,49.38,105.93,80.56
4,8,91.32,93.64,95.00,107.41,73.89
5,7,95.00,92.58,93.12,97.78,68.06
6,8,95.05,102.99,56.25,99.07,50.00
7,7,72.85,86.85,60.00,0.00,56.11
8,8,84.26,93.10,47.50,18.52,50.83
9,7,90.10,97.55,51.25,88.89,63.61


Note that the `inplace` attribute causes pandas to fill the values inline and does not return a copy of the dataframe, but instead modifies the dataframe you have.

In addition to rules controlling how missing values might be loaded, it's sometimes useful to consider missing values as actually having information. For example, let's assume you have data on logs from some online video sharing website. Let's say you have a backend system that keeps logs of the video usage. In these systems it's common for the player to have a heartbeat functionality where playback statistics are sent to the server every so often, maybe every 30 seconds. These heartbeats can get big as they can carry the whole state of the playback system such as where the video play head is at, what the video size is, which video is being rendered to the screen, how loud the volume is, and more.

In [ ]:
# If we load the data file log.csv, we can see an example of what this might look like.
df = pd.read_csv("/content/drive/My Drive/Applied Data Science in Python/datasets/log.csv")
df.head(10)

,time,user,video,playback position,paused,volume
0,1469974424,cheryl,intro.html,5,False,10.0
1,1469974454,cheryl,intro.html,6,NaN,NaN
2,1469974544,cheryl,intro.html,9,NaN,NaN
3,1469974574,cheryl,intro.html,10,NaN,NaN
4,1469977514,bob,intro.html,1,NaN,NaN
5,1469977544,bob,intro.html,1,NaN,NaN
6,1469977574,bob,intro.html,1,NaN,NaN
7,1469977604,bob,intro.html,1,NaN,NaN
8,1469974604,cheryl,intro.html,11,NaN,NaN
9,1469974694,cheryl,intro.html,14,NaN,NaN


In this data the first column is a timestamp in the Unix epoch format. The next column is the user name followed by the web page they're visiting and the video that they're playing. Each row of the DataFrame has a playback position. We can see that as the playback position increases by one, the time stamp increases by about 30 seconds...

...except for user Bob. It turns out that Bob has paused his playback so as time increases the playback position doesn't change. Note too how difficult it is for us to try and derive this knowledge from the data, because it's not sorted by time stamp as one might expect. This is actually not uncommon on systems which have a high degree of parallelism. Note also, that there are a lot of missing values in the paused and volume columns. This is because it is not efficient to send this information across the network if it hasn't changed. So what this particular system does is insert null values into the database if there are no changes.

To see things clearer, let's sort the dataframe by timestamp. In Pandas we can do that by sorting either by index or by values. First, we promote the time stamp to an index using `set_index()` and then sort on the index using `sort_index()`.

In [ ]:
df = df.set_index('time')
df = df.sort_index()
df.head(20)

,user,video,playback position,paused,volume
time,,,,,
1469974424,cheryl,intro.html,5,False,10.0
1469974424,sue,advanced.html,23,False,10.0
1469974454,cheryl,intro.html,6,NaN,NaN
1469974454,sue,advanced.html,24,NaN,NaN
1469974484,cheryl,intro.html,7,NaN,NaN
1469974514,cheryl,intro.html,8,NaN,NaN
1469974524,sue,advanced.html,25,NaN,NaN
1469974544,cheryl,intro.html,9,NaN,NaN
1469974554,sue,advanced.html,26,NaN,NaN


If we look closely at the output though we'll notice that the index isn't really unique. This is because two users can use the system at the same time-- again, a very common case. So, a more correct approach is to reset the index, and use some multi-level indexing on time AND user together instead, and promote the user name to a second level of the index to deal with that issue.

In [ ]:
df = df.reset_index()
df = df.set_index(['time', 'user'])
df

video  playback position paused  volume
time       user                                                   
1469974424 cheryl     intro.html                  5  False    10.0
           sue     advanced.html                 23  False    10.0
1469974454 cheryl     intro.html                  6    NaN     NaN
           sue     advanced.html                 24    NaN     NaN
1469974484 cheryl     intro.html                  7    NaN     NaN
1469974514 cheryl     intro.html                  8    NaN     NaN
1469974524 sue     advanced.html                 25    NaN     NaN
1469974544 cheryl     intro.html                  9    NaN     NaN
1469974554 sue     advanced.html                 26    NaN     NaN
1469974574 cheryl     intro.html                 10    NaN     NaN
1469974604 cheryl     intro.html                 11    NaN     NaN
1469974624 sue     advanced.html                 27    NaN     NaN
1469974634 cheryl     intro.html                 12    NaN     NaN
1469974654 sue     advanced.html                 28    NaN     5.0
1469974664 cheryl     intro.html                 13    NaN     NaN
1469974694 cheryl     intro.html                 14    NaN     NaN
1469974724 cheryl     intro.html                 15    NaN     NaN
           sue     advanced.html                 29    NaN     NaN
1469974754 sue     advanced.html                 30    NaN     NaN
1469974824 sue     advanced.html                 31    NaN     NaN
1469974854 sue     advanced.html                 32    NaN     NaN
1469974924 sue     advanced.html                 33    NaN     NaN
1469977424 bob        intro.html                  1   True    10.0
1469977454 bob        intro.html                  1    NaN     NaN
1469977484 bob        intro.html                  1    NaN     NaN
1469977514 bob        intro.html                  1    NaN     NaN
1469977544 bob        intro.html                  1    NaN     NaN
1469977574 bob        intro.html                  1    NaN     NaN
1469977604 bob        intro.html                  1    NaN     NaN
1469977634 bob        intro.html                  1    NaN     NaN
1469977664 bob        intro.html                  1    NaN     NaN
1469977694 bob        intro.html                  1    NaN     NaN
1469977724 bob        intro.html                  1    NaN     NaN

One of the parameters in the `.fillna()` attribute is the `method` parameter. The `method` parameter has two common fill values:

1. `"ffill"`: forward fills and updates any `NA` value of a particular cell with the value from the previous row. 
2. `"bfill"`: backward fills, which is the opposite of `"ffill"`. It fills the missing values with the next valid value. 

It's important to note that *your data needs to be sorted in order for this to have the effect you might want*. Data coming from traditional database management systems usually have no order guarantee, just like our log data. So be careful.

But, since we now have the data indexed and sorted appropriately, we can fill the missing data using `"ffill"`. 

Note: It's good to remember when dealing with missing values that you can deal with individual columns or sets of columns by projecting them. So you don't have to fix all missing values in one command.

In [ ]:
df = df.fillna(method='ffill')
df

,time,user,video,playback position,paused,volume
0,1469974424,cheryl,intro.html,5,False,10.0
1,1469974454,cheryl,intro.html,6,False,10.0
2,1469974544,cheryl,intro.html,9,False,10.0
3,1469974574,cheryl,intro.html,10,False,10.0
4,1469977514,bob,intro.html,1,False,10.0
5,1469977544,bob,intro.html,1,False,10.0
6,1469977574,bob,intro.html,1,False,10.0
7,1469977604,bob,intro.html,1,False,10.0
8,1469974604,cheryl,intro.html,11,False,10.0
9,1469974694,cheryl,intro.html,14,False,10.0


We can also do customized fill-in to replace values with the `replace()` function. It allows replacement from several approaches: value-to-value, list, dictionary, and regex. 

Let's generate a simple example.

In [ ]:
df = pd.DataFrame({'A': [1, 1, 2, 3, 4],
                   'B': [3, 6, 3, 8, 9],
                   'C': ['a', 'b', 'c', 'd', 'e']})
df

,A,B,C
0,1,3,a
1,1,6,b
2,2,3,c
3,3,8,d
4,4,9,e


Let's say we want to replace 1's with 100. We can do that using the **value-to-value approach**.


In [ ]:
df.replace(1, 100)

,A,B,C
0,100,3,a
1,100,6,b
2,2,3,c
3,3,8,d
4,4,9,e


What if we want to change two values? Here, we can try the **list approach**.

For example, let's change 1's to 100 and 3's to 300.

In [ ]:
df.replace([1, 3], [100, 300])

,A,B,C
0,100,300,a
1,100,6,b
2,2,300,c
3,300,8,d
4,4,9,e


What's really cool about pandas replacement is that it supports regex too! 

Let's look at our data from the logs dataset again and try the **regex approach**.

In [ ]:
df = pd.read_csv("/content/drive/My Drive/Applied Data Science in Python/datasets/log.csv")
df.head(10)

,time,user,video,playback position,paused,volume
0,1469974424,cheryl,intro.html,5,False,10.0
1,1469974454,cheryl,intro.html,6,NaN,NaN
2,1469974544,cheryl,intro.html,9,NaN,NaN
3,1469974574,cheryl,intro.html,10,NaN,NaN
4,1469977514,bob,intro.html,1,NaN,NaN
5,1469977544,bob,intro.html,1,NaN,NaN
6,1469977574,bob,intro.html,1,NaN,NaN
7,1469977604,bob,intro.html,1,NaN,NaN
8,1469974604,cheryl,intro.html,11,NaN,NaN
9,1469974694,cheryl,intro.html,14,NaN,NaN


The `replace()` function has parameters that can allow us to replace using a regex.

1. The first parameter `to_replace` takes the regex pattern we want to match.
2. The second parameter `value` take the value we want to emit upon match.
3. The third parameter `regex`, which a boolean parameter. If set to `True`, would treat the string passed on `to_replace` as a regular expression.

Take a moment to think about this problem: imagine we want to detect all html pages in the "video" column, and we want to overwrite them with the keyword "webpage". How could we accomplish this?

In [ ]:
# Here's my solution, match any number of characters that end in .html
df.replace(to_replace=".*\.html$", value="webpage", regex=True)

,time,user,video,playback position,paused,volume
0,1469974424,cheryl,webpage,5,False,10.0
1,1469974454,cheryl,webpage,6,NaN,NaN
2,1469974544,cheryl,webpage,9,NaN,NaN
3,1469974574,cheryl,webpage,10,NaN,NaN
4,1469977514,bob,webpage,1,NaN,NaN
5,1469977544,bob,webpage,1,NaN,NaN
6,1469977574,bob,webpage,1,NaN,NaN
7,1469977604,bob,webpage,1,NaN,NaN
8,1469974604,cheryl,webpage,11,NaN,NaN
9,1469974694,cheryl,webpage,14,NaN,NaN


One last note on missing values. When you use statistical functions on DataFrames, these functions typically ignore missing values. For instance if you try and calculate the mean value of a DataFrame, the underlying NumPy function will ignore missing values. This is usually what you want but you should be aware that values are being excluded. 

Why you have missing values really matters depending upon the problem you are trying to solve. It might be unreasonable to infer missing values, for instance, if the data shouldn't exist in the first place.

-----
#Data Manipulation
-----

Data manipulation refers to the process of adjusting the data to make it organized and easier to read. It is a big part of Data Science, and something you will be spending time doing with any new dataset you lay your hands on. 

To introduce you to data manipulation, let's start with a basic data cleaning process example and take a look at a few more pandas API functions.

In [ ]:
# Let's start by bringing in pandas...
import pandas as pd
# ...and load our dataset. We're going to be cleaning the list of presidents in the US from wikipedia
df=pd.read_csv("/content/drive/My Drive/Applied Data Science in Python/datasets/presidents.csv")

df.head()

,#,President,Born,Age atstart of presidency,Age atend of presidency,Post-presidencytimespan,Died,Age
0,1,George Washington,"Feb 22, 1732[a]","57 years, 67 daysApr 30, 1789","65 years, 10 daysMar 4, 1797","2 years, 285 days","Dec 14, 1799","67 years, 295 days"
1,2,John Adams,"Oct 30, 1735[a]","61 years, 125 daysMar 4, 1797","65 years, 125 daysMar 4, 1801","25 years, 122 days","Jul 4, 1826","90 years, 247 days"
2,3,Thomas Jefferson,"Apr 13, 1743[a]","57 years, 325 daysMar 4, 1801","65 years, 325 daysMar 4, 1809","17 years, 122 days","Jul 4, 1826","83 years, 82 days"
3,4,James Madison,"Mar 16, 1751[a]","57 years, 353 daysMar 4, 1809","65 years, 353 daysMar 4, 1817","19 years, 116 days","Jun 28, 1836","85 years, 104 days"
4,5,James Monroe,"Apr 28, 1758","58 years, 310 daysMar 4, 1817","66 years, 310 daysMar 4, 1825","6 years, 122 days","Jul 4, 1831","73 years, 67 days"


Let's look at the data. We have some presidents, some dates, I see a bunch of footnotes in the "Born" column which might cause issues, and more. 

Say I want to start by cleaning up the President name into firstname and lastname. Take a minute, and think about how would you do that?

There are numerous ways to tackle this. Here are a few:

1. Using a regex. We can create two new columns and apply a regex to the projection of the "President" column.


In [ ]:
# Make a copy of the President column
df["First"]=df['President']
# Then we can call replace() and just have a pattern that matches the last name and set it to an empty string
df["First"]=df["First"].replace("[ ].*", "", regex=True)
#repeat the same for the Last name
df["Last"]=df['President']
df["Last"]=df["Last"].replace(".*[ ]", "", regex=True)
# Now let's take a look
df.head(10)

,#,President,Born,Age atstart of presidency,Age atend of presidency,Post-presidencytimespan,Died,Age,First,Last
0,1,George Washington,"Feb 22, 1732[a]","57 years, 67 daysApr 30, 1789","65 years, 10 daysMar 4, 1797","2 years, 285 days","Dec 14, 1799","67 years, 295 days",George,Washington
1,2,John Adams,"Oct 30, 1735[a]","61 years, 125 daysMar 4, 1797","65 years, 125 daysMar 4, 1801","25 years, 122 days","Jul 4, 1826","90 years, 247 days",John,Adams
2,3,Thomas Jefferson,"Apr 13, 1743[a]","57 years, 325 daysMar 4, 1801","65 years, 325 daysMar 4, 1809","17 years, 122 days","Jul 4, 1826","83 years, 82 days",Thomas,Jefferson
3,4,James Madison,"Mar 16, 1751[a]","57 years, 353 daysMar 4, 1809","65 years, 353 daysMar 4, 1817","19 years, 116 days","Jun 28, 1836","85 years, 104 days",James,Madison
4,5,James Monroe,"Apr 28, 1758","58 years, 310 daysMar 4, 1817","66 years, 310 daysMar 4, 1825","6 years, 122 days","Jul 4, 1831","73 years, 67 days",James,Monroe
5,6,John Quincy Adams,"Jul 11, 1767","57 years, 236 daysMar 4, 1825","61 years, 236 daysMar 4, 1829","18 years, 356 days","Feb 23, 1848","80 years, 227 days",John,Adams
6,7,Andrew Jackson,"Mar 15, 1767","61 years, 354 daysMar 4, 1829","69 years, 354 daysMar 4, 1837","8 years, 96 days","Jun 8, 1845","78 years, 85 days",Andrew,Jackson
7,8,Martin Van Buren,"Dec 5, 1782","54 years, 89 daysMar 4, 1837","58 years, 89 daysMar 4, 1841","21 years, 142 days","Jul 24, 1862","79 years, 231 days",Martin,Buren
8,9,William H. Harrison,"Feb 9, 1773","68 years, 23 daysMar 4, 1841","68 years, 54 days Apr 4, 1841[b]",NaN,"Apr 4, 1841","68 years, 54 days",William,Harrison
9,10,John Tyler,"Mar 29, 1790","51 years, 6 daysApr 4, 1841","54 years, 340 daysMar 4, 1845","16 years, 320 days","Jan 18, 1862","71 years, 295 days",John,Tyler


That works... but it's slow, since we had to make a full copy of a column then go through and update strings. There are a few other ways we can deal with this. The most general one is...  

2. using the [`apply()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html) function. The `apply()` function on a pandas dataframe takes some arbitrary function you have written and applies it to either a Series (a single column) or DataFrame across all rows or columns. 

In [ ]:
# Let's drop the columns we made first
del df["First"]
del df["Last"]

# Lets write a function which splits a string into two pieces using a single row of data
def splitname(row):
    # Let's extract the firstname and create a new entry in the series
    row['First']=row['President'].split(" ")[0]
    # Let's do the same with the last word in the string
    row['Last']=row['President'].split(" ")[-1]
    # Now we just return the row and the pandas .apply() will take of merging them back into a DataFrame
    return row

# Now we "apply" this to the dataframe indicating we want to apply the function across each row
# The row is a single Series object which is a single row indexed by column values
df=df.apply(splitname, axis='columns')
df.head(10)

,#,President,Born,Age atstart of presidency,Age atend of presidency,Post-presidencytimespan,Died,Age,First,Last
0,1,George Washington,"Feb 22, 1732[a]","57 years, 67 daysApr 30, 1789","65 years, 10 daysMar 4, 1797","2 years, 285 days","Dec 14, 1799","67 years, 295 days",George,Washington
1,2,John Adams,"Oct 30, 1735[a]","61 years, 125 daysMar 4, 1797","65 years, 125 daysMar 4, 1801","25 years, 122 days","Jul 4, 1826","90 years, 247 days",John,Adams
2,3,Thomas Jefferson,"Apr 13, 1743[a]","57 years, 325 daysMar 4, 1801","65 years, 325 daysMar 4, 1809","17 years, 122 days","Jul 4, 1826","83 years, 82 days",Thomas,Jefferson
3,4,James Madison,"Mar 16, 1751[a]","57 years, 353 daysMar 4, 1809","65 years, 353 daysMar 4, 1817","19 years, 116 days","Jun 28, 1836","85 years, 104 days",James,Madison
4,5,James Monroe,"Apr 28, 1758","58 years, 310 daysMar 4, 1817","66 years, 310 daysMar 4, 1825","6 years, 122 days","Jul 4, 1831","73 years, 67 days",James,Monroe
5,6,John Quincy Adams,"Jul 11, 1767","57 years, 236 daysMar 4, 1825","61 years, 236 daysMar 4, 1829","18 years, 356 days","Feb 23, 1848","80 years, 227 days",John,Adams
6,7,Andrew Jackson,"Mar 15, 1767","61 years, 354 daysMar 4, 1829","69 years, 354 daysMar 4, 1837","8 years, 96 days","Jun 8, 1845","78 years, 85 days",Andrew,Jackson
7,8,Martin Van Buren,"Dec 5, 1782","54 years, 89 daysMar 4, 1837","58 years, 89 daysMar 4, 1841","21 years, 142 days","Jul 24, 1862","79 years, 231 days",Martin,Buren
8,9,William H. Harrison,"Feb 9, 1773","68 years, 23 daysMar 4, 1841","68 years, 54 days Apr 4, 1841[b]",NaN,"Apr 4, 1841","68 years, 54 days",William,Harrison
9,10,John Tyler,"Mar 29, 1790","51 years, 6 daysApr 4, 1841","54 years, 340 daysMar 4, 1845","16 years, 320 days","Jan 18, 1862","71 years, 295 days",John,Tyler


It's pretty questionable as to whether that the best way, but it achieves the result and I find that I use the `apply()` function regularly in my code. The pandas series has a couple of other nice convenient functions though, which brings us to third way we can also achieve the same results...

3. the [`.extract()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.extract.html) function. The extract takes a regular expression (regex) as input and specifically requires you to set capture groups that correspond to the output columns you are interested in.

Take a second to reflect on this - if you were going to write a regular expression that returned groups and just had the firstname and lastname in it, what would that look like?

In [ ]:
#Lets drop our firstname and lastname columns again and try again.
del df['First']
del df['Last']

# Here's my solution, we capture two groups containing the first and the last name
pattern="(^[\w]*)\s*\w*\.*\s+([\w]*$)"

# The extract function is built into the str attribute of the Series object, 
# so we can call it using Series.str.extract(pattern)
df["President"].str.extract(pattern).head(10)

,0,1
0,George,Washington
1,John,Adams
2,Thomas,Jefferson
3,James,Madison
4,James,Monroe
5,John,Adams
6,Andrew,Jackson
7,Martin,Buren
8,William,Harrison
9,John,Tyler


In [ ]:
# So that looks pretty nice, other than the column names. But if we name the groups we get named columns out
pattern="(?P<First>^[\w]*)\s*\w*\.*\s+(?P<Last>[\w]*$)"

# Now call extract
names=df["President"].str.extract(pattern).head(10)
names

,First,Last
0,George,Washington
1,John,Adams
2,Thomas,Jefferson
3,James,Madison
4,James,Monroe
5,John,Adams
6,Andrew,Jackson
7,Martin,Buren
8,William,Harrison
9,John,Tyler


In [ ]:
# And we can just copy these into our main dataframe if we want to
df["First"]=names["First"]
df["Last"]=names["Last"]
df.head()

,#,President,Born,Age atstart of presidency,Age atend of presidency,Post-presidencytimespan,Died,Age,First,Last
0,1,George Washington,"Feb 22, 1732[a]","57 years, 67 daysApr 30, 1789","65 years, 10 daysMar 4, 1797","2 years, 285 days","Dec 14, 1799","67 years, 295 days",George,Washington
1,2,John Adams,"Oct 30, 1735[a]","61 years, 125 daysMar 4, 1797","65 years, 125 daysMar 4, 1801","25 years, 122 days","Jul 4, 1826","90 years, 247 days",John,Adams
2,3,Thomas Jefferson,"Apr 13, 1743[a]","57 years, 325 daysMar 4, 1801","65 years, 325 daysMar 4, 1809","17 years, 122 days","Jul 4, 1826","83 years, 82 days",Thomas,Jefferson
3,4,James Madison,"Mar 16, 1751[a]","57 years, 353 daysMar 4, 1809","65 years, 353 daysMar 4, 1817","19 years, 116 days","Jun 28, 1836","85 years, 104 days",James,Madison
4,5,James Monroe,"Apr 28, 1758","58 years, 310 daysMar 4, 1817","66 years, 310 daysMar 4, 1825","6 years, 122 days","Jul 4, 1831","73 years, 67 days",James,Monroe


It's worth looking at the pandas `str` module for other functions which have been written specifically to clean up strings in DataFrames. You can find these functions in the Working with Text section: https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html

Now lets move on to clean up that Born column. First, let's get rid of anything that isn't in the pattern of Month, Day and Year.

In [ ]:
df["Born"]=df["Born"].str.extract("([\w]{3} [\w]{1,2}, [\w]{4})")
df["Born"].head()

0    Feb 22, 1732
1    Oct 30, 1735
2    Apr 13, 1743
3    Mar 16, 1751
4    Apr 28, 1758
Name: Born, dtype: object

So, that cleans up the date format. But before moving on, I want to foreshadow something else here - and that is the type of this column.  

The type of this column is object, and we know that's what pandas uses when it is dealing with string. However, pandas actually has really interesting date/time features - in fact, that's one of the reasons Wes McKinney put his efforts into the library, to be able to deal with financial transactions. So if I were building this out, I would actually update this column to the correct data type as well -- `datetime64`. We do this using the [`to_datetime()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html) function.

This would make subsequent processing on the dataframe around dates, such as getting every President who was born in a given time span, much easier.

In [ ]:
df["Born"]=pd.to_datetime(df["Born"])
df["Born"].head()

0   1732-02-22
1   1735-10-30
2   1743-04-13
3   1751-03-16
4   1758-04-28
Name: Born, dtype: datetime64[ns]

Now, most of the other columns in this dataset I would clean in a similar fashion. This would be a good practice activity for you, so I would recommend that you try to finish cleaning up this dataframe when you have the time. 

Also note that I introduced you to the `str` module which has a number of important functions for cleaning pandas dataframes. However, you don't have to use these - I actually use `apply()` quite a bit myself, especially if I don't need high performance data cleaning because my dataset is small. But, it's worth noting that the `str` functions are incredibly useful and build on your existing knowledge of regular expressions. They are also vectorized and efficient to use as well. 

##Merging DataFrames

Next, let's look at how we can bring multiple dataframe objects together, either by merging them horizontally, or by concatenating them vertically. 

However, before we jump into the code, we need to address a little relational theory and to get some language conventions down.

Let's start by looking at a "Venn Diagram", which is traditionally used to show set membership. For example, here we have two populations. The circle on the left is the population of students at a university, and the circle on the right is the population of the staff at a university. The overlapping region in the middle are all of those students who are also staff; i.e. students that run tutorials for a course, or grade assignments, or engage in running research experiments.

![Venn Diagram](https://drive.google.com/uc?id=1PZGq_1sKq5lZ_VGlVomG-a0IXAfjJCS1)



When it comes to translating this to pandas, this is akin to having these two 
populations as two separate DataFrames, maybe with a common label called "Person Name". When we want to join the DataFrames together, we have some choices to make:

1. Do we want a list of all the people regardless of whether they're staff or student, and all of the information we can get on them? In database terminology, this is called a **full outer join**. And in set theory, it's called a **union**. In the Venn diagram, it would represent everyone in any circle. Here's an image of what that would look like in the Venn diagram:


![Union](https://drive.google.com/uc?id=1fXyraAjCLW9_3tBif7HmA44qvtyfaa9H)

2. Or do we only want those people who we have maximum information for; i.e.people who are both staff and students. For example, maybe being a staff member and a student involves getting a tuition waiver, and we want to calculate the cost of this. In database terminology, this is called an **inner join**. Or in set theory, the **intersection**. It is represented in the Venn diagram as the overlapping parts of each circle.

![Intersection](https://drive.google.com/uc?id=1nyB5aCGjUr_OFgVs_fZeH0DKuyYJZ6KM)


With that background, let's see an example of how we would do this in pandas using the `merge()` function.

In [ ]:
import pandas as pd

# First we create two DataFrames, staff and students.
staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR'},
                         {'Name': 'Sally', 'Role': 'Course liasion'},
                         {'Name': 'James', 'Role': 'Grader'}])
# And let's index these staff by name
staff_df = staff_df.set_index('Name')
# Now we'll create a student dataframe
student_df = pd.DataFrame([{'Name': 'James', 'School': 'Business'},
                           {'Name': 'Mike', 'School': 'Law'},
                           {'Name': 'Sally', 'School': 'Engineering'}])
# And we'll index this by name too
student_df = student_df.set_index('Name')

Let's take a look at what they look like...

In [ ]:
staff_df

,Role
Name,
Kelly,Director of HR
Sally,Course liasion
James,Grader


In [ ]:
student_df

,School
Name,
James,Business
Mike,Law
Sally,Engineering


There's some overlap in these DataFrames in that James and Sally are both students and staff, but Mike and Kelly are not. Importantly, both DataFrames are indexed along the value we want to merge them on: `Name`.

Now, If we want the union of these, we would call [`merge()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html) and pass in the DataFrame on the left and the DataFrame on the right. We would then tell `merge()` that we want it to use an outer join, and want to use the left and right indices as the joining columns.

In [ ]:
pd.merge(staff_df, student_df, how='outer', left_index=True, right_index=True)

,Role,School
Name,,
James,Grader,Business
Kelly,Director of HR,NaN
Mike,NaN,Law
Sally,Course liasion,Engineering


As you can see, everyone is listed in the resulting DataFrame. However, since Mike does not have a role, and John does not have a school, those cells are listed as missing values.

If we wanted to get the intersection, that is, just those who are a student AND a staff, we could set the `how` attribute to `'inner'`. Again, we set both left and right indices to be true as the joining columns.

In [ ]:
pd.merge(staff_df, student_df, how='inner', left_index=True, right_index=True)

,Role,School
Name,,
Sally,Course liasion,Engineering
James,Grader,Business


And we see the resulting DataFrame has only James and Sally in it. 

Now, there are two other common use cases when merging DataFrames. The first is when we would want to get a list of all staff regardless of whether they were students or not. But if they were students, we would want to get their student details as well. To do this we would use a **left join**. 

It is important to note the order of dataframes in this function: the first dataframe is the left dataframe and the second is the right.

In [ ]:
pd.merge(staff_df, student_df, how='left', left_index=True, right_index=True)

,Role,School
Name,,
Kelly,Director of HR,NaN
Sally,Course liasion,Engineering
James,Grader,Business


Alternatively, if we want a list of all of the students and their roles if they were also staff, we would use a **right join**.

In [ ]:
pd.merge(staff_df, student_df, how='right', left_index=True, right_index=True)

,Role,School
Name,,
James,Grader,Business
Mike,NaN,Law
Sally,Course liasion,Engineering


We can also do it another way. The merge method has a couple of other interesting parameters. First, you don't need to use indices to join on, you can use columns as well. 

To do that, we use the `merge()` parameter `on`, and we can assign a column (instead of indices) that both dataframe has as the joining column

In [ ]:
# First, lets remove the index from both of our dataframes
staff_df = staff_df.reset_index()
student_df = student_df.reset_index()

# Now lets merge using the on parameter
pd.merge(staff_df, student_df, how='right', on='Name')

,Name,Role,School
0,James,Grader,Business
1,Mike,NaN,Law
2,Sally,Course liasion,Engineering


Personally, I find myself using the `on` parameter instead of a the index when using `merge()` the most.

Let's recreate new staff and student DataFrames that have a location information added to them.

In [ ]:
staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR', 
                          'Location': 'State Street'},
                         {'Name': 'Sally', 'Role': 'Course liasion', 
                          'Location': 'Washington Avenue'},
                         {'Name': 'James', 'Role': 'Grader', 
                          'Location': 'Washington Avenue'}])

student_df = pd.DataFrame([{'Name': 'James', 'School': 'Business', 
                            'Location': '1024 Billiard Avenue'},
                           {'Name': 'Mike', 'School': 'Law', 
                            'Location': 'Fraternity House #22'},
                           {'Name': 'Sally', 'School': 'Engineering', 
                            'Location': '512 Wilson Crescent'}])

staff_df

,Name,Role,Location
0,Kelly,Director of HR,State Street
1,Sally,Course liasion,Washington Avenue
2,James,Grader,Washington Avenue


In [ ]:
student_df

,Name,School,Location
0,James,Business,1024 Billiard Avenue
1,Mike,Law,Fraternity House #22
2,Sally,Engineering,512 Wilson Crescent


So what happens when we have conflicts between the DataFrames? (Both have a column labeled `Location`)

In the staff DataFrame, this is an office location, but in the student DataFrame, the location information is actually their home address.

The merge function preserves this information, but appends an `_x` or `_y` to help differentiate between which index went with which column of data. The `_x` is always the left DataFrame information, and the `_y` is always the right DataFrame information.

Here, if we want all the staff information regardless of whether they were students or not, but if they were students, we would want to get their student details as well, we can do a left join and on the column of `Name`

In [ ]:
pd.merge(staff_df, student_df, how='left', on='Name')

,Name,Role,Location_x,School,Location_y
0,Kelly,Director of HR,State Street,NaN,NaN
1,Sally,Course liasion,Washington Avenue,Engineering,512 Wilson Crescent
2,James,Grader,Washington Avenue,Business,1024 Billiard Avenue


From the output, we can see there are columns `Location_x` and `Location_y`. `Location_x` refers to the Location column in the left dataframe, which is staff dataframe and `Location_y` refers to the Location column in the right dataframe, which is student dataframe.

Before we leave merging of DataFrames, let's talk about multi-indexing and multiple columns. It's quite possible that the first name for students and staff might overlap, but the last name might not. In this case, we use a list of the multiple columns that should be used to join keys from both dataframes on the `on` parameter. Remember though that the column name(s) assigned to the `on` parameter needs to exist in both dataframes.

In [ ]:
# Let's regernate new student and staff data
staff_df = pd.DataFrame([{'First Name': 'Kelly', 'Last Name': 'Desjardins', 
                          'Role': 'Director of HR'},
                         {'First Name': 'Sally', 'Last Name': 'Brooks', 
                          'Role': 'Course liasion'},
                         {'First Name': 'James', 'Last Name': 'Wilde', 
                          'Role': 'Grader'}])
student_df = pd.DataFrame([{'First Name': 'James', 'Last Name': 'Hammond', 
                            'School': 'Business'},
                           {'First Name': 'Mike', 'Last Name': 'Smith', 
                            'School': 'Law'},
                           {'First Name': 'Sally', 'Last Name': 'Brooks', 
                            'School': 'Engineering'}])

staff_df

,First Name,Last Name,Role
0,Kelly,Desjardins,Director of HR
1,Sally,Brooks,Course liasion
2,James,Wilde,Grader


In [ ]:
student_df

,First Name,Last Name,School
0,James,Hammond,Business
1,Mike,Smith,Law
2,Sally,Brooks,Engineering


As you see here, James Wilde and James Hammond don't match on both keys since they have different last names. So we should expect that an inner join doesn't include these individuals in the output, and only Sally Brooks will be retained.

In [ ]:
pd.merge(staff_df, student_df, how='inner', on=['First Name','Last Name'])

,First Name,Last Name,Role,School
0,Sally,Brooks,Course liasion,Engineering


Joining dataframes through merging is incredibly common, and you'll need to know how to pull data from different sources, clean it, and join it for analysis. This is a staple not only of pandas, but of database technologies as well.

##Concatenating DataFrames

If we think of merging as joining "horizontally", meaning we join on similar values in a column found in two dataframes, then concatenating is joining "vertically", meaning we put dataframes on top of each other.

Let's understand this from an example. Let's assume we have a dataset that tracks some information over the years. Each year's record is a separate CSV and every CSV for every year's record has *the exact same columns*. What happens if we want to put all the data, from all years' records, together? We can **concatenate** them.

Let's take a look at the US Department of Education College Scorecard [data](https://data.ed.gov/dataset/college-scorecard-all-data-files-through-6-2020/resources). It has each US university's data on student completion, student debt, after-graduation income, and more. The data is stored in separate CSV's with each CSV containing a year's record. For your convenience, I downloaded the records from 2016 to 2018 and truncated them to the first 200 rows only in each file so the files are easier to upload. 

Say we want to concatenate those three years into one dataframe. We start by creating three dataframes, each containing one year's record.

In [ ]:
df_2016 = pd.read_csv("/content/drive/My Drive/Applied Data Science in Python/datasets/college_scorecard/MERGED2015_16_PP_first200.csv")
df_2017 = pd.read_csv("/content/drive/My Drive/Applied Data Science in Python/datasets/college_scorecard/MERGED2016_17_PP_first200.csv")
df_2018 = pd.read_csv("/content/drive/My Drive/Applied Data Science in Python/datasets/college_scorecard/MERGED2017_18_PP_first200.csv")

In [ ]:
# Let's get a view of one of the dataframes
df_2016.head(10)

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,SCH_DEG,HCM2,MAIN,NUMBRANCH,PREDDEG,HIGHDEG,CONTROL,ST_FIPS,REGION,LOCALE,LOCALE2,LATITUDE,LONGITUDE,CCBASIC,CCUGPROF,CCSIZSET,HBCU,PBI,ANNHI,TRIBAL,AANAPII,HSI,NANTI,MENONLY,WOMENONLY,RELAFFIL,ADM_RATE,ADM_RATE_ALL,SATVR25,SATVR75,...,PLUS_DEBT_ALL_COMP_MDPAY10_SUPP,PLUS_DEBT_INST_NOCOMP_N,PLUS_DEBT_INST_NOCOMP_MD,PLUS_DEBT_ALL_NOCOMP_N,PLUS_DEBT_ALL_NOCOMP_MD,PLUS_DEBT_INST_MALE_N,PLUS_DEBT_INST_MALE_MD,PLUS_DEBT_ALL_MALE_N,PLUS_DEBT_ALL_MALE_MD,PLUS_DEBT_INST_NOMALE_N,PLUS_DEBT_INST_NOMALE_MD,PLUS_DEBT_ALL_NOMALE_N,PLUS_DEBT_ALL_NOMALE_MD,PLUS_DEBT_INST_PELL_N,PLUS_DEBT_INST_PELL_MD,PLUS_DEBT_ALL_PELL_N,PLUS_DEBT_ALL_PELL_MD,PLUS_DEBT_INST_NOPELL_N,PLUS_DEBT_INST_NOPELL_MD,PLUS_DEBT_ALL_NOPELL_N,PLUS_DEBT_ALL_NOPELL_MD,PLUS_DEBT_INST_STAFFTHIS_N,PLUS_DEBT_INST_STAFFTHIS_MD,PLUS_DEBT_ALL_STAFFTHIS_N,PLUS_DEBT_ALL_STAFFTHIS_MD,PLUS_DEBT_INST_NOSTAFFTHIS_N,PLUS_DEBT_INST_NOSTAFFTHIS_MD,PLUS_DEBT_ALL_NOSTAFFTHIS_N,PLUS_DEBT_ALL_NOSTAFFTHIS_MD,PLUS_DEBT_INST_STAFFANY_N,PLUS_DEBT_INST_STAFFANY_MD,PLUS_DEBT_ALL_STAFFANY_N,PLUS_DEBT_ALL_STAFFANY_MD,PLUS_DEBT_INST_NOSTAFFANY_N,PLUS_DEBT_INST_NOSTAFFANY_MD,PLUS_DEBT_ALL_NOSTAFFANY_N,PLUS_DEBT_ALL_NOSTAFFANY_MD,COUNT_NWNE_3YR,COUNT_WNE_3YR,CNTOVER150_3YR
0,100654,100200,1002,Alabama A & M University,Normal,AL,35762,NaN,NaN,NaN,3.0,NaN,1,1,3,4,1,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6538,0.6538,383.0,470.0,...,NaN,629,10078,724,10065,468,10114,521,10078,468,10698,589,10963,777,10000,919,10000,159,16547,191,16547,914,10378,1073,10500,22,10906,37,9609,915,10406,1089,10500,21,10812,21,10812,NaN,NaN,NaN
1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,NaN,NaN,NaN,3.0,NaN,1,1,3,4,1,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6043,0.6043,520.0,630.0,...,NaN,361,12591,800,12353,259,12520,574,13303,522,12080,1109,12190,448,10000,936,9700,333,17264,747,18032,770,PrivacySuppressed,1429,12068,11,PrivacySuppressed,254,16460,PrivacySuppressed,PrivacySuppressed,1665,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,18,PrivacySuppressed,NaN,NaN,NaN
2,100690,2503400,25034,Amridge University,Montgomery,AL,36117-3553,NaN,NaN,NaN,3.0,NaN,1,1,3,4,2,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,PrivacySuppressed,PrivacySuppressed,41,6000,PrivacySuppressed,PrivacySuppressed,20,6805,PrivacySuppressed,PrivacySuppressed,46,6250,PrivacySuppressed,PrivacySuppressed,55,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,11,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,NaN,NaN,NaN
3,100706,105500,1055,University of Alabama in Huntsville,Huntsville,AL,35899,NaN,NaN,NaN,3.0,NaN,1,1,3,4,1,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.8120,0.8120,520.0,650.0,...,NaN,131,12133,254,11289,139,12000,258,12114,143,13000,255,10754,178,10452,311,9490,104,15910,202,15272,PrivacySuppressed,PrivacySuppressed,439,11600,PrivacySuppressed,PrivacySuppressed,74,12002,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,NaN,NaN,NaN
4,100724,100500,1005,Alabama State University,Montgomery,AL,36104-0271,NaN,NaN,NaN,3.0,NaN,1,1,3,4,1,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.4639,0.4639,370.0,450.0,...,NaN,611,9660,673,9816,386,10523,428,10658,545,10230,619,10132,806,9867,897,9844,125,12880,150,14744,919,PrivacySuppressed,1021,10522,12,PrivacySuppressed,26,10210,920,PrivacySuppressed,1035,PrivacySuppressed,11,PrivacySuppressed,12,PrivacySuppressed,NaN,NaN,NaN
5,100751,105100,1051,The University of Alabama,Tusca

We see that there is a whopping number of columns - more than 2000! We can also check the length of each dataframe as well.

In [ ]:
print(len(df_2016))
print(len(df_2017))
print(len(df_2018))

200
200
200


Let's just put all three dataframes in a list and call that list `frames` and pass the list into the `concat()` function. Let's see what it looks like.


In [ ]:
frames = [df_2016, df_2017, df_2018]
pd.concat(frames)

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,SCH_DEG,HCM2,MAIN,NUMBRANCH,PREDDEG,HIGHDEG,CONTROL,ST_FIPS,REGION,LOCALE,LOCALE2,LATITUDE,LONGITUDE,CCBASIC,CCUGPROF,CCSIZSET,HBCU,PBI,ANNHI,TRIBAL,AANAPII,HSI,NANTI,MENONLY,WOMENONLY,RELAFFIL,ADM_RATE,ADM_RATE_ALL,SATVR25,SATVR75,...,PLUS_DEBT_ALL_COMP_MDPAY10_SUPP,PLUS_DEBT_INST_NOCOMP_N,PLUS_DEBT_INST_NOCOMP_MD,PLUS_DEBT_ALL_NOCOMP_N,PLUS_DEBT_ALL_NOCOMP_MD,PLUS_DEBT_INST_MALE_N,PLUS_DEBT_INST_MALE_MD,PLUS_DEBT_ALL_MALE_N,PLUS_DEBT_ALL_MALE_MD,PLUS_DEBT_INST_NOMALE_N,PLUS_DEBT_INST_NOMALE_MD,PLUS_DEBT_ALL_NOMALE_N,PLUS_DEBT_ALL_NOMALE_MD,PLUS_DEBT_INST_PELL_N,PLUS_DEBT_INST_PELL_MD,PLUS_DEBT_ALL_PELL_N,PLUS_DEBT_ALL_PELL_MD,PLUS_DEBT_INST_NOPELL_N,PLUS_DEBT_INST_NOPELL_MD,PLUS_DEBT_ALL_NOPELL_N,PLUS_DEBT_ALL_NOPELL_MD,PLUS_DEBT_INST_STAFFTHIS_N,PLUS_DEBT_INST_STAFFTHIS_MD,PLUS_DEBT_ALL_STAFFTHIS_N,PLUS_DEBT_ALL_STAFFTHIS_MD,PLUS_DEBT_INST_NOSTAFFTHIS_N,PLUS_DEBT_INST_NOSTAFFTHIS_MD,PLUS_DEBT_ALL_NOSTAFFTHIS_N,PLUS_DEBT_ALL_NOSTAFFTHIS_MD,PLUS_DEBT_INST_STAFFANY_N,PLUS_DEBT_INST_STAFFANY_MD,PLUS_DEBT_ALL_STAFFANY_N,PLUS_DEBT_ALL_STAFFANY_MD,PLUS_DEBT_INST_NOSTAFFANY_N,PLUS_DEBT_INST_NOSTAFFANY_MD,PLUS_DEBT_ALL_NOSTAFFANY_N,PLUS_DEBT_ALL_NOSTAFFANY_MD,COUNT_NWNE_3YR,COUNT_WNE_3YR,CNTOVER150_3YR
0,100654,100200,1002,Alabama A & M University,Normal,AL,35762,NaN,NaN,NaN,3.0,NaN,1,1,3,4,1,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6538,0.6538,383.0,470.0,...,NaN,629,10078,724,10065,468,10114,521,10078,468,10698,589,10963,777,10000,919,10000,159,16547,191,16547,914,10378,1073,10500,22,10906,37,9609,915,10406,1089,10500,21,10812,21,10812,NaN,NaN,NaN
1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,NaN,NaN,NaN,3.0,NaN,1,1,3,4,1,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6043,0.6043,520.0,630.0,...,NaN,361,12591,800,12353,259,12520,574,13303,522,12080,1109,12190,448,10000,936,9700,333,17264,747,18032,770,PrivacySuppressed,1429,12068,11,PrivacySuppressed,254,16460,PrivacySuppressed,PrivacySuppressed,1665,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,18,PrivacySuppressed,NaN,NaN,NaN
2,100690,2503400,25034,Amridge University,Montgomery,AL,36117-3553,NaN,NaN,NaN,3.0,NaN,1,1,3,4,2,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,PrivacySuppressed,PrivacySuppressed,41,6000,PrivacySuppressed,PrivacySuppressed,20,6805,PrivacySuppressed,PrivacySuppressed,46,6250,PrivacySuppressed,PrivacySuppressed,55,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,11,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,NaN,NaN,NaN
3,100706,105500,1055,University of Alabama in Huntsville,Huntsville,AL,35899,NaN,NaN,NaN,3.0,NaN,1,1,3,4,1,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.8120,0.8120,520.0,650.0,...,NaN,131,12133,254,11289,139,12000,258,12114,143,13000,255,10754,178,10452,311,9490,104,15910,202,15272,PrivacySuppressed,PrivacySuppressed,439,11600,PrivacySuppressed,PrivacySuppressed,74,12002,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,NaN,NaN,NaN
4,100724,100500,1005,Alabama State University,Montgomery,AL,36104-0271,NaN,NaN,NaN,3.0,NaN,1,1,3,4,1,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.4639,0.4639,370.0,450.0,...,NaN,611,9660,673,9816,386,10523,428,10658,545,10230,619,10132,806,9867,897,9844,125,12880,150,14744,919,PrivacySuppressed,1021,10522,12,PrivacySuppressed,26,10210,920,PrivacySuppressed,1035,PrivacySuppressed,11,PrivacySuppressed,12,PrivacySuppressed,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,

As you can see, we now have more observations in the resulting dataframe, however, columns remained the same. If we scroll down to the bottom of the output, we see that there are a total of 600 rows after concatenating the three dataframes, which is the total number of rows in all 3 dataframes.


However, now that all the data is concatenated together, we don't know what observations are from what year anymore! 

Actually, the [concat()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html) function has a parameter that solves such a problem: the `keys` parameter. The `keys` parameter helps you set an extra level of indices. This is done by passing in a list of keys that you want to correspond to the dataframes into the `keys` parameter

In [ ]:
# Now let's try it out
pd.concat(frames, keys=['2016','2017','2018'])

UNITID    OPEID  ...  COUNT_WNE_3YR CNTOVER150_3YR
2016 0    100654   100200  ...            NaN            NaN
     1    100663   105200  ...            NaN            NaN
     2    100690  2503400  ...            NaN            NaN
     3    100706   105500  ...            NaN            NaN
     4    100724   100500  ...            NaN            NaN
...          ...      ...  ...            ...            ...
2018 195  109721   902200  ...          321.0          172.0
     196  109730  2553500  ...          145.0           70.0
     197  109785   111700  ...          891.0          773.0
     198  109819   111800  ...           84.0           72.0
     199  109907   111900  ...           28.0           22.0

[600 rows x 2044 columns]

Now, we have the indices as the year so we know what observations are from what year. You should know that concatenation also has inner and outer methods. If you are concatenating two dataframes that do not have identical columns, and choose the outer method, some cells will be `NaN`. If you choose to do inner, then some observations will be dropped due to `NaN` values. You can think of this as analogous to the left and right joins of the `merge()` function.

Now you know how to merge and concatenate datasets together. You will find such functions very useful for combining data needed to do more complex analysis. A solid understanding of how to merge data is absolutely essential when you are procuring, cleaning, and manipulating data. It's worth knowing how to merge different datasets quickly, and the different options you can use when merging datasets, and I would encourage you to check out the pandas docs for merging and concatenating data.